In [6]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from time import time
from keras.callbacks import ModelCheckpoint
import cv2
import random, os
import numpy as np

directorySeparator = "\\"

testImages = "E:\\594_data\\smoothmuscle_NN_Data\\test_holdout\\images"

pericyteMasks = "E:\\594_data\\\pericyte_NN_Data\\test_holdout\\masks" 
smoothmuscleMasks = "E:\\594_data\\smoothmuscle_NN_Data\\test_holdout\\masks" 

outputDir = "E:\\594_data\\confusionMasks"

window_size = 65
middle = int(window_size / 2)

In [7]:
def mirror_image(img):
    height, width = img.shape

    #the following creates a mirrored image with the edges mirrored using a size window_size

    newImg = np.zeros((height+(window_size*2),width+(window_size*2)), np.uint8)
    for h in range(height):
        for w in range(width):
            newImg[h+window_size,w+window_size] = img[h,w]

    for w in range(window_size):
        for h in range(height):
            newImg[h+window_size,w]=img[h,window_size-1-w]
            newImg[h+window_size,width+(2*window_size)-1-w]=img[h,width-(window_size-w)]

    for h in range(window_size):
        for w in range(width):
            newImg[h,w+window_size]=img[window_size-1-h,w]
            newImg[height+(2*window_size)-1-h,w+window_size]=img[height-(window_size-h),w]
            
    return newImg
    
                
    
    

In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(1,65,65 ), data_format='channels_first'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


model.load_weights("weights.balanced.hdf5") #best weights for the cell/no-cell classifier?
#model.load_weights("weights.best.imbalanced.hdf5")
print("loaded weights from disk")

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy'])


model2 = Sequential()
model2.add(Conv2D(32, (3, 3), input_shape=(1,65,65 ), data_format='channels_first'))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Conv2D(32, (3, 3)))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Conv2D(64, (3, 3)))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model2.add(Dense(64))
model2.add(Activation('relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))


model2.load_weights("weights.distinguisher.hdf5")

print("loaded weights from disk")

model2.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy'])



loaded weights from disk
loaded weights from disk


In [9]:
totalPericyte = 0
totalSmoothmuscle = 0
totalNoCell = 0

totalEverything = 0


In [13]:


for filename in os.listdir(testImages): #for each file in the test images
    if filename.endswith(".tif"): 
        
        print("Filename: "+ filename)
        
        img = cv2.imread(testImages+directorySeparator+filename,-1)
        img = (img/256).astype('uint8')
        mirror = mirror_image(img)
        output_img = np.zeros(img.shape)
        
        output_img = np.stack((output_img,)*3, axis=-1)
        
        print(output_img.shape)
        
        
        
        pericyteMask = cv2.imread(pericyteMasks+directorySeparator+filename,-1)
    
        smoothmuscleMask = cv2.imread(smoothmuscleMasks+directorySeparator+filename,-1)
    
        height,width = img.shape
        
        for h in range(int(height)): #for each pixel in the image
            for w in range(width):
                
                isPericyte = False
                isSmoothMuscle = False
                isNoCell = False
                
                if (pericyteMask[h,w]==255):
                    isPericyte = True
                    totalPericyte += 1
                    #print("pericyte present at: " + str(h) + "," +str(w))
                
                elif (smoothmuscleMask[h,w]==255):
                    isSmoothMuscle = True
                    totalSmoothmuscle += 1
                    #print("smooth muscle present at: " + str(h) + "," +str(w))
                else:
                    isNoCell = True
                    totalNoCell += 1
                    
                totalEverything += 1
                    
                
                
                
                crop_img = mirror[(h+window_size-middle):(h+window_size+middle+1), (w+window_size-middle):(w+window_size+middle+1)]
                #test_array.append([crop_img / (1./255)])
                
                crop_img = [[crop_img] ]
                crop_img = np.array(crop_img)
                crop_img = crop_img * (1./255)
                #print(crop_img.shape)
                
                cellExistPrediction = model.predict(crop_img)
                
                if (cellExistPrediction[0] < 0.005 ): #if the pixel is classified as having a cell
                    cellTypePrediction = model2.predict(crop_img)
                    
                    if (cellTypePrediction < .5): #0 is pericyte, 1 is smoothmuscle, so pericyte first
                        if isPericyte:
                            output_img[h][w][0] = 0 
                            output_img[h][w][1] = 255
                            output_img[h][w][2] = 0
                        if isSmoothMuscle:
                            output_img[h][w][0] = 255 
                            output_img[h][w][1] = 0
                            output_img[h][w][2] = 0
                        if isNoCell:
                            output_img[h][w][0] = 255
                            output_img[h][w][1] = 51
                            output_img[h][w][2] = 153
                    
                    else: #smooth muscle
                        if isPericyte:
                            output_img[h][w][0] = 255
                            output_img[h][w][1] = 255
                            output_img[h][w][2] = 0
                        if isSmoothMuscle:
                            output_img[h][w][0] = 0
                            output_img[h][w][1] = 0
                            output_img[h][w][2] = 255
                        if isNoCell:
                            output_img[h][w][0] = 102
                            output_img[h][w][1] = 51
                            output_img[h][w][2] = 0                  
                        #print("smooth muscle detected at: " + str(h) + "," +str(w))
                
                else: #pixel has no cell
                    if isPericyte:
                            output_img[h][w][0] = 102
                            output_img[h][w][1] = 0
                            output_img[h][w][2] = 51
                    if isSmoothMuscle:
                            output_img[h][w][0] = 153
                            output_img[h][w][1] = 0
                            output_img[h][w][2] = 0
                    if isNoCell:
                            output_img[h][w][0] = 255
                            output_img[h][w][1] = 255
                            output_img[h][w][2] = 255
                    
        cv2.imwrite(outputDir+directorySeparator+filename,output_img)


Filename: Amy_4th_Z71-80_2nd_cropped0007.tif
(512, 512, 3)
Filename: Amy_4th_Z71-80_2nd_cropped0018.tif
(512, 512, 3)


KeyboardInterrupt: 